[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ryanfobel/gridwatch-history/main)

# Download operational data about the Ontario grid

We can use this data to reverse-engineer the models used by gridwatch.ca and co2signal.com (e.g., see [issue #1](https://github.com/ryanfobel/gridwatch-history/issues/1) on github).

In [280]:
%load_ext autoreload
%autoreload 2

import os
import sys
import json

sys.path.insert(0, os.path.join(".."))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

from pipelines.gridwatch import load_file

%matplotlib inline

rcParams.update({"figure.figsize": (12, 6)})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [366]:
from pipelines.ieso.CA import get_historical_file_list_pre2019

historical_files = get_historical_file_list_pre2019()
historical_files

ImportError: cannot import name 'get_historical_file_list_pre2019'

In [254]:
from pipelines.ieso.CA import fetch_production, fetch_production_by_fuel, fetch_price
import arrow

now = arrow.now()

In [255]:
data = fetch_price(target_datetime=now)
print(len(data))
data

13


[{'datetime': datetime.datetime(2023, 4, 29, 1, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400), 'UTC-5')),
  'price': 5.57,
  'currency': 'CAD',
  'source': 'ieso.ca',
  'zoneKey': 'CA-ON'},
 {'datetime': datetime.datetime(2023, 4, 29, 2, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400), 'UTC-5')),
  'price': 5.92,
  'currency': 'CAD',
  'source': 'ieso.ca',
  'zoneKey': 'CA-ON'},
 {'datetime': datetime.datetime(2023, 4, 29, 3, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400), 'UTC-5')),
  'price': 6.08,
  'currency': 'CAD',
  'source': 'ieso.ca',
  'zoneKey': 'CA-ON'},
 {'datetime': datetime.datetime(2023, 4, 29, 4, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400), 'UTC-5')),
  'price': 9.5,
  'currency': 'CAD',
  'source': 'ieso.ca',
  'zoneKey': 'CA-ON'},
 {'datetime': datetime.datetime(2023, 4, 29, 5, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400), 'UTC-5')),
  'price': 9.55,
  'currency': 'CAD',
  'source': 'ieso.ca',
  'zoneKey': '

In [256]:
# 3-months of historical data available via IESO xml feed [1]
# 1. https://github.com/electricitymaps/electricitymaps-contrib/blob/master/parsers/CA_ON.py)
data = fetch_production(target_datetime=now.shift(months=-3).datetime)
print(len(data))
data

4320


,name,fuel,dt,production
0,BRUCEA-G1,nuclear,2023-01-29 01:00:00-05:00,821.0
1,BRUCEA-G1,nuclear,2023-01-29 02:00:00-05:00,820.0
2,BRUCEA-G1,nuclear,2023-01-29 03:00:00-05:00,820.0
3,BRUCEA-G1,nuclear,2023-01-29 04:00:00-05:00,821.0
4,BRUCEA-G1,nuclear,2023-01-29 05:00:00-05:00,823.0
...,...,...,...,...
4315,TBAYBOWATER CTS,biomass,2023-01-29 20:00:00-05:00,0.0
4316,TBAYBOWATER CTS,biomass,2023-01-29 21:00:00-05:00,0.0
4317,TBAYBOWATER CTS,biomass,2023-01-29 22:00:00-05:00,0.0
4318,TBAYBOWATER CTS,biomass,2023-01-29 23:00:00-05:00,0.0


In [257]:
data_json = fetch_production_by_fuel(target_datetime=now)
data_json[-1]['datetime'] = data_json[-1]['datetime'].isoformat()
print(json.dumps(data_json[-1], indent=4))

{
    "datetime": "2023-04-29T13:00:00-05:00",
    "zoneKey": "CA-ON",
    "production": {
        "biomass": 0.0,
        "gas": 893.0,
        "hydro": 5806.0,
        "nuclear": 7883.0,
        "solar": 99.0,
        "wind": 1324.0
    },
    "storage": {},
    "source": "ieso.ca"
}


In [260]:
data = fetch_production(target_datetime=now)

output = pd.DataFrame()
for timestamp in data["dt"].unique():
    row = data[data["dt"] == timestamp][
        ["name", "production"]
    ].set_index("name").T
    row["datetime"] = timestamp
    row = row.set_index("datetime")
    output.columns.name = ""
    output = pd.concat([
        output,
        row
    ], axis=0)

output
    

,BRUCEA-G1,BRUCEA-G2,BRUCEA-G3,BRUCEA-G4,BRUCEB-G5,BRUCEB-G6,BRUCEB-G7,BRUCEB-G8,DARLINGTON-G1,DARLINGTON-G2,...,GRANDSF,KINGSTONSF,NANTICOKE SOLAR,NORTHLAND POWER SOLAR FACILITIES,SOUTHGATE SF,STONE MILLS SF,WINDSOR AIRPORT SF,ATIKOKAN-G1,CALSTOCKGS,TBAYBOWATER CTS
datetime,,,,,,,,,,,,,,,,,,,,,
2023-04-29 01:00:00-05:00,820.0,825.0,0.0,0.0,817.0,0.0,824.0,803.0,0.0,885.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-29 02:00:00-05:00,820.0,824.0,0.0,0.0,817.0,0.0,824.0,804.0,0.0,884.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-29 03:00:00-05:00,820.0,824.0,0.0,0.0,816.0,0.0,824.0,804.0,0.0,884.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-29 04:00:00-05:00,820.0,824.0,0.0,0.0,816.0,0.0,824.0,804.0,0.0,884.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-29 05:00:00-05:00,819.0,824.0,0.0,0.0,815.0,0.0,824.0,804.0,0.0,884.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-29 06:00:00-05:00,819.0,824.0,0.0,0.0,815.0,0.0,824.0,803.0,0.0,884.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-29 07:00:00-05:00,819.0,824.0,0.0,0.0,815.0,0.0,824.0,803.0,0.0,885.0,...,1.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0
2023-04-29 08:00:00-05:00,819.0,824.0,0.0,0.0,816.0,0.0,824.0,804.0,0.0,885.0,...,7.0,4.0,3.0,9.0,2.0,2.0,2.0,0.0,0.0,0.0
2023-04-29 09:00:00-05:00,819.0,824.0,0.0,0.0,816.0,0.0,824.0,806.0,0.0,884.0,...,14.0,10.0,5.0,14.0,5.0,4.0,3.0,0.0,0.0,0.0


![biomass](../images/co2signal.com/biomass.png)
![coal](../images/co2signal.com/coal.png)
![gas](../images/co2signal.com/gas.png)
![geothermal](../images/co2signal.com/geothermal.png)
![hydro](../images/co2signal.com/hydro.png)
![nuclear](../images/co2signal.com/nuclear.png)
![oil](../images/co2signal.com/oil.png)
![solar](../images/co2signal.com/solar.png)
![wind](../images/co2signal.com/wind.png)

In [11]:
df_intensity = pd.DataFrame({
    "biomass": {
        "carbon_intensity": 230,
        "source": "IPCC 2014"
    },
    "coal": {
        "carbon_intensity": 820,
        "source": "IPCC 2014"
    },
    "gas": {
        "carbon_intensity": 490,
        "source": "IPCC 2014"
    },
    "geothermal": {
        "carbon_intensity": 38,
        "source": "IPCC 2014"
    },
    "hydro": {
        "carbon_intensity": 24,
        "source": "IPCC 2014"
    },
    "nuclear": {
        "carbon_intensity": 5,
        "source": "Mallia, E., Lewis, G. 'Life cycle greenhouse gas emissions of electricity generation in the province of Ontario, Canada' (2013)"
    },
    "oil": {
        "carbon_intensity": 650,
        "source": "UK POST 2014"
    },
    "solar": {
        "carbon_intensity": 45,
        "source": "IPCC 2014"
    },
    "wind": {
        "carbon_intensity": 11,
        "source": "IPCC 2014"
    },
})
df_intensity

,biomass,coal,gas,geothermal,hydro,nuclear,oil,solar,wind
carbon_intensity,230,820,490,38,24,5,650,45,11
source,IPCC 2014,IPCC 2014,IPCC 2014,IPCC 2014,IPCC 2014,"Mallia, E., Lewis, G. 'Life cycle greenhouse g...",UK POST 2014,IPCC 2014,IPCC 2014


In [12]:
sources = [col[len("production."):] for col in df.columns if col.startswith("production.")]

df["production.total"] = 0
for source in sources:
    df["production.total"] = df["production.total"] + df[f"production.{source}"]

df["emissions.total"] = 0
for source in sources:
    df[f"emissions.{source}"] = df[f"production.{source}"] * df_intensity[source].carbon_intensity
    df["emissions.total"] = df["emissions.total"] + df[f"emissions.{source}"]
df

,filepath,filename,year,month,url,production.total,emissions.total
0,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201905.csv,2019,05,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
1,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201906.csv,2019,06,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
2,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201907.csv,2019,07,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
3,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201908.csv,2019,08,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
4,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201909.csv,2019,09,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
5,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201910.csv,2019,10,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
6,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201911.csv,2019,11,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
7,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201912.csv,2019,12,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
8,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_202001.csv,2020,01,http://reports.ieso.ca/public/GenOutputCapabil...,0,0
9,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_202002.csv,2020,02,http://reports.ieso.ca/public/GenOutputCapabil...,0,0


In [13]:
df["carbon_intensity"] = df["emissions.total"] / df["production.total"] # kg/MWh or g/kWh

# production in MW, emissions in kg CO2e, carbon intensity (gCO2e/kWh)
df[["production.total", "emissions.total", "carbon_intensity"]]

,production.total,emissions.total,carbon_intensity
0,0,0,NaN
1,0,0,NaN
2,0,0,NaN
3,0,0,NaN
4,0,0,NaN
5,0,0,NaN
6,0,0,NaN
7,0,0,NaN
8,0,0,NaN
9,0,0,NaN
